In [38]:
import torch
from torch.utils.data import DataLoader
import os
from utils import *
import matplotlib.pyplot as plt
import numpy as np
import yaml
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix

In [39]:
# please make sure that CONFIG is the same one used to generate the loaded model
CONFIG = 'bert_1.yml'

In [40]:
def pick_device(config):
    if config['device'] == 'gpu':
        if torch.cuda.is_available():
            device_type = "cuda"
        else:
            device_type = "cpu"
        print(f"using device type: {device_type}")
        device = torch.device(device_type)
    return device

In [41]:
# load data
with open(os.path.join('configs', CONFIG),'r') as file:
        config = yaml.safe_load(file)
print('config:')
print(config)

device = pick_device(config)
torch.cuda.empty_cache()

tokenizer, model, optimizer = load_model(config['bert_type'], float(config['learning_rate']), device)

data = load_data('data', config['data_file'])
train_data_loader, test_data_loader = process_data(data, tokenizer, test_size=config['test_data_pct'], max_len=config['max_len'], batch_size=config['batch_size'])

config:
{'test_data_pct': 0.8, 'learning_rate': '1e-3', 'bert_type': 'tinybert', 'epochs': 15, 'batch_size': 64, 'max_len': 200, 'device': 'gpu', 'data_file': 'shuffled_data.csv'}
using device type: cpu


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [42]:
# metrics scores
def metrics_scores(y_pred, y_true):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)

    return accuracy, recall, precision, f1, confusion

In [48]:
# load trained model .pth 
# please adjust the path of the .pth file
model_pthFile = '.../models/1689776953_8841338.pth'
loaded_state = torch.load(model_pthFile)
loaded_state_dict = loaded_state.state_dict()

if isinstance(loaded_state_dict, dict):
    print("loaded_state_dict is dictionary-like.")
else:
    print("loaded_state_dict is not dictionary-like.")

model.load_state_dict(loaded_state_dict)

model.eval()

loaded_state_dict is dictionary-like.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [50]:
def evaluate(model, data_loader, device):
    model_eval = model.eval()
    correct = 0
    total = 0

    predictions = []
    labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            local_labels = batch["labels"].to(device)

            outputs = model_eval(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, local_predictions = torch.max(outputs.logits, dim=1)

            correct += torch.sum(local_predictions == local_labels)
            total += local_labels.shape[0]

            predictions.extend(local_predictions)
            labels.extend(local_labels)
            
            
    accuracy = correct/total
    
    return accuracy.item()

In [51]:
accuracies = {}
accuracies['initial test'] = evaluate(model, test_data_loader, device)

In [52]:
print(accuracies['initial test'])

0.9544666409492493


In [ ]:
''''
# not completed, still working on

# SHARP, heat map 

from captum.attr import GradientShap

# Create a GradientShap instance
gradient_shap = GradientShap(model)

reference_scores = []
y_pred = []

saliency_values_list = []
for inputs, targets in test_data_loader:
    # compute saliency values
    saliency_values = gradient_shap.attribute(X_val)

    # convert saliency values to absolute values
    saliency_values_abs = torch.abs(saliency_values)

    # heat map
    saliency_heatmap = saliency_values_abs.sum(dim=1)
    
    saliency_values_list.append(saliency_heatmap.detach().numpy())

saliency_mean = np.mean(saliency_values_list)

plt.imshow(saliency_mean, cmap = 'hot', alpha=0.8)   
plt.show()
'''